<a href="https://colab.research.google.com/github/illusoryTwin/InnoML/blob/main/task2/ML_task2_3004.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


# Normalize pixel values of the images
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


170498071/170498071 [==============================] - 4s 0us/step


In [13]:
from tensorflow.keras import models, layers

import cv2
# Define a function to apply blur to images

# Define data augmentation parameters including the blur
datagen = ImageDataGenerator(
    shear_range=0.1,  # Shear transformation
    rotation_range=15,  # Rotation
    channel_shift_range=0.1,  # Color filtering
)



# # Define data augmentation parameters
# datagen = ImageDataGenerator(
#     shear_range=0.1,  # Shear transformation
#     rotation_range=15,  # Rotation
#     # width_shift_range=0.1,  # Width shift
#     # height_shift_range=0.1,  # Height shift
#     # zoom_range=0.1,  # Zoom
#     channel_shift_range=0.1,  # Color filtering
#     # horizontal_flip=True,
#     # fill_mode='nearest'
# )

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])


In [16]:
import numpy as np
from tensorflow.keras.callbacks import Callback

class CosineAnnealingScheduler(Callback):
    def __init__(self, lr_min, lr_max, T, T_mul=1):
        super(CosineAnnealingScheduler, self).__init__()
        self.lr_min = lr_min
        self.lr_max = lr_max
        self.T = T
        self.T_mul = T_mul
        self.cycle = 0

    def on_epoch_begin(self, epoch, logs=None):
        if epoch == 0:
            self.cycle = 0
        else:
            self.cycle += 1
            if self.cycle % self.T == 0:
                self.lr_max *= 0.5
                self.cycle = 0
                self.T *= self.T_mul

        lr = self.lr_min + 0.5 * (self.lr_max - self.lr_min) * (1 + np.cos(np.pi * self.cycle / self.T))
        self.model.optimizer.lr = lr

# Define initial learning rate and maximum learning rate
lr_min = 0.001
lr_max = 0.01

# Define number of epochs for each cycle and multiplier for each cycle
T = 10
T_mul = 2

# Create the CosineAnnealingScheduler
cosine_annealing_scheduler = CosineAnnealingScheduler(lr_min, lr_max, T, T_mul)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with data augmentation and the cosine annealing scheduler
history = model.fit(datagen.flow(x_train, y_train, batch_size=256),
                    epochs=100,
                    validation_data=(x_test, y_test),
                    callbacks=[early_stopping, cosine_annealing_scheduler])

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test loss: {test_loss}, Test accuracy: {test_acc}')


Epoch 1/100
196/196 [==============================] - 39s 173ms/step - loss: 0.5937 - accuracy: 0.7955 - val_loss: 1.8728 - val_accuracy: 0.6101
Epoch 2/100
196/196 [==============================] - 33s 170ms/step - loss: 0.5493 - accuracy: 0.8102 - val_loss: 0.8298 - val_accuracy: 0.7610
Epoch 3/100
196/196 [==============================] - 34s 171ms/step - loss: 0.5174 - accuracy: 0.8205 - val_loss: 0.7220 - val_accuracy: 0.7712
Epoch 4/100
196/196 [==============================] - 35s 178ms/step - loss: 0.4633 - accuracy: 0.8394 - val_loss: 0.6023 - val_accuracy: 0.8147
Epoch 5/100
196/196 [==============================] - 33s 166ms/step - loss: 0.4282 - accuracy: 0.8511 - val_loss: 0.5308 - val_accuracy: 0.8336
Epoch 6/100
196/196 [==============================] - 34s 171ms/step - loss: 0.3763 - accuracy: 0.8689 - val_loss: 0.5224 - val_accuracy: 0.8332
Epoch 7/100
196/196 [==============================] - 34s 172ms/step - loss: 0.3321 - accuracy: 0.8845 - val_loss: 0.5087 -

In [17]:
import numpy as np
from tensorflow.keras.utils import to_categorical

# Load test images and labels
test_images = np.load("task_2_test_images.npy")
test_labels = np.load("task_2_test_labels.npy")

# Normalize pixel values
test_images = test_images.astype('float32') / 255.0
# Convert test labels to one-hot encoded format (if needed)
test_labels = to_categorical(test_labels, num_classes=10)

# Evaluate the model on the test dataset
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test loss: {test_loss}, Test accuracy: {test_acc}')


312/312 [==============================] - 2s 5ms/step - loss: 1.0987 - accuracy: 0.7086
Test loss: 1.098737120628357, Test accuracy: 0.7086179256439209


In [18]:
import numpy as np
from sklearn.metrics import roc_auc_score

# Predict probabilities for the test set
y_pred_probs = model.predict(test_images)

# Calculate AUC-ROC score
auc_roc = roc_auc_score(test_labels, y_pred_probs)

print(f'AUC-ROC score: {auc_roc}')


312/312 [==============================] - 1s 3ms/step
AUC-ROC score: 0.953846314498565


# Transfer learning

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.utils.multiclass import unique_labels

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

#Import standard libraries

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline
''' to learn more about itertools visit
    https://medium.com/@jasonrigden/a-guide-to-python-itertools-82e5a306cdf8'''
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

#Import keras functions

from keras import Sequential

'''Since we are using transfer learning let's import the model that we want to implement.Let's use VGG 19(19 layers) and Resnet-50 (50 layers of residual units).
Residual units allow us to add more layers onto the model without a degradation in accuracy.
Let's try and compare the accuracy of the 2 models and see if the addtional layers do make a significant difference. '''

from keras.applications import VGG19,ResNet50

'Import the datagenerator to augment images'
from keras.preprocessing.image import ImageDataGenerator

'''Import the optimizers and leanring rate annealer (which will reduce the learning rate once a particular metric we choose(in this case validation error)
does not reduce after a user defined number of epochs)'''
from keras.optimizers import SGD,Adam
from keras.callbacks import ReduceLROnPlateau

'Lastly import the final layers that will be added on top of the base model'
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout

'Import to_categorical from the keras utils package to one hot encode the labels'
from keras.utils import to_categorical

#Import dataset
from keras.datasets import cifar10

#Divide the data in Train, Validation and Test Datasets
'I had to turn the Internet setting to on to download load the dataset'
(x_train,y_train),(x_test,y_test)=cifar10.load_data()

x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=.3)

In [ ]:
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

In [ ]:
y_train=to_categorical(y_train)
y_val=to_categorical(y_val)
y_test=to_categorical(y_test)

# Lets print the dimensions one more time to see if things changed the way we expected

print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))



In [ ]:
train_generator = ImageDataGenerator(
                                    rotation_range=2,
                                    horizontal_flip=True,
                                    zoom_range=.1 )

val_generator = ImageDataGenerator(
                                    rotation_range=2,
                                    horizontal_flip=True,
                                    zoom_range=.1)

test_generator = ImageDataGenerator(
                                    rotation_range=2,
                                    horizontal_flip= True,
                                    zoom_range=.1)

#Fit the augmentation method to the data

train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

